## ЦЕЛЬ  
Подготовить основу рекомендательной системы.  

## ЗАДАЧИ  
Оформить таблицу , которая станет  основой рекомендательной  системы и должна содержать три столбца:  
1.Курс к котрому идет рекомендация  
2.Курс для рекомендации №1 (самый популярный)  
3.Курс для рекомендации №2 (второй по популярности)  

## КОНКРЕТНЫЕ ШАГИ (ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ)
1.Познакомиться с датасетом  
2.подготовить и проанализировать данные с помощью SQL.  
3.Обработать данные средствами Python.  
4.Составить итоговую таблицу с рекомендациями, снабдив её необходимыми комментариями, и представить отчёт продакт-менеджеру.


## sql запросы в Metabase
Сколько клиентов покупали курсы?  

SELECT COUNT(DISTINCT c.user_id)   
FROM  final.carts c   
join final.cart_items t on c.id =  t.cart_id  
where resource_type = 'Course' and state = 'successful'  


Сколько всего есть различных курсов?  

SELECT   
count (distinct t.resource_id)  
FROM  final.carts c   
join final.cart_items t on c.id =  t.cart_id  
where resource_type = 'Course'  

Каково среднее число купленных курсов на одного клиента?  

SELECT   
    count (distinct c.user_id) as unique_users,  
    count (distinct t.resource_id) as unique_courses,  
    count (t.resource_id) as total_sales_courses,  
    count (t.resource_id) / count (distinct c.user_id)::decimal as average_by_users  
FROM  final.carts c   
join final.cart_items t on c.id =  t.cart_id  
where resource_type = 'Course'and c.state = 'successful'  

Сколько клиентов купили больше одного курса?  

WITH count_users AS  
(  
    SELECT  user_id,  
            COUNT (resource_id) AS count_cours  
    FROM final.carts AS c  
    JOIN final.cart_items AS ci  
    ON ci.cart_id = c.id  
    WHERE state = 'successful'  
    AND resource_type = 'Course'  
    GROUP BY  user_id  
    HAVING COUNT (DISTINCT resource_id) > 1  
)  
SELECT    
DISTINCT count_users.user_id,  
resource_id  
FROM final.carts AS c  
JOIN final.cart_items AS ci  
ON ci.cart_id = c.id  
RIGHT JOIN count_users  
ON c.user_id = count_users.user_id  
WHERE state = 'successful' AND resource_type = 'Course'  
ORDER BY count_users.user_id, resource_id  


Переносим данные и Metabase в VSCode

In [2]:
# импортирую необходимые библиоткеки
import pandas as pd
from scipy import stats as st
import itertools as it
import numpy as np
import collections 
from collections import Counter

In [3]:
# загружаю csv файл
df= pd.read_csv('C:\IDE\DATA\query_result_2024-03-28T12_07_24.038744Z.csv')

In [4]:
# Проверяю на пропуски
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34074 entries, 0 to 34073
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      34074 non-null  int64
 1   resource_id  34074 non-null  int64
dtypes: int64(2)
memory usage: 532.5 KB


In [5]:
#основной датасет перевожу в df с user_id и списком купленных курсов к каждому  
df_dict = df.groupby('user_id')['resource_id'].agg(
        ['nunique', set]
).sort_values(by='nunique', ascending=False)

# формирую пары для каждого пользьзователя
course_pairs= df_dict['set'].apply(lambda x: list(it.combinations(sorted(x), 2)))

# объединяю получившиеся списки пар курсов в один
courses_list = []
for course in course_pairs.values:
    for elem in course: 
        courses_list.append(elem) 
# считаю кол-во уникальных пар
print('Кол-во уникальных пар : ', len(set(courses_list)))


Кол-во уникальных пар :  3989


In [6]:
# Определяю самую популярную пару курсов
pairs= pd.DataFrame(courses_list)
pairs[0]=pairs[0].astype(str)
pairs[1]=pairs[1].astype(str)

pairs['pairs'] = pairs[[0,1]].agg(', '.join,axis=1)
display(pairs['pairs'].value_counts())

pairs
551, 566     797
515, 551     417
489, 551     311
523, 551     304
566, 794     290
            ... 
571, 866       1
368, 679       1
835, 1102      1
809, 1125      1
358, 1103      1
Name: count, Length: 3989, dtype: int64

In [7]:
# Создаю словарь из courses_list для дальнейших расчетов
Counter(courses_list)
courses_pairs_dict=dict(Counter(courses_list))

In [8]:
## Формирую перечень уникальных resource_id
courses_unique = set(df['resource_id'])
# создаю функцию для вывода рекомендаций
def reco(course_id):
    top_list = []
    for e in courses_pairs_dict.keys():
        if e[0] == course_id:
            top_list.append((e, courses_pairs_dict[e]))
        if e[1] == course_id:
            top_list.append((e, courses_pairs_dict[e]))
    top_list.sort(key = lambda x: x[1], reverse=True) 
    return top_list[:2]


Считаю минимальную границу частотности  

Процентиль беру 85, т.к. он поможет определить какие значения находятся в верхних 15%  и поможет выделить лучшие результаты,соответствено поможет выделить самые популярные курсы и на основе их предложить клиентам  более интересные им  курсы  и тем самым повышать средний чек

In [9]:

freq_df = pd.DataFrame(courses_pairs_dict, index = courses_pairs_dict.keys(), columns = ['freq'])
freq_df['freq'] = courses_pairs_dict.values()
min_freq = np.percentile(freq_df['freq'],85)

In [10]:
# Формирую перечень уникальных resource_id, создаю список
courses_unique = list(set(df['resource_id']))

# Создаю  датафрейм с индексами из resource_id, полученных выше
recomm_df = pd.DataFrame(index=courses_unique,columns=["first_rec","second_rec"])

# Добавляю значения в recomm_df
for course_id in courses_unique:
        rec1= None
        rec2= None
        if reco(course_id)[0][1] >= min_freq:
            rec1 = (set(reco(course_id)[0][0])-set([course_id])).pop()
        else:
            rec1 = '511'
        if reco(course_id)[1][1] >= min_freq:
            rec2 = (set(reco(course_id)[1][0])-set([course_id])).pop()
        else:
            rec2 = '566'
        recomm_df.loc[course_id] = [rec1,rec2]
   

In [11]:
# Проверяю правильность работы функции
reco(566)

[((551, 566), 797), ((566, 794), 290)]

In [12]:
# вывожу итоговую таблицу
recomm_df

,first_rec,second_rec
513,503,551
514,551,515
515,551,489
516,745,553
517,551,750
...,...,...
504,572,569
507,570,752
508,507,570
509,745,553


Таблица выдает две рекомендации для курса, первая- самый популярный курс, вторая- второй по популярности курс, благодаря этим рекомендациям, клиентам будут предлагатьсяся курсы наиболее интересные для них, а при увеличении интереса клиентов будут возрастать продажи и средний чек.